In [1]:
import pandas as pd
import numpy as np
import shapely
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, concatenate, Dropout, UpSampling2D, Reshape
gpu_devices = tf.config.experimental.list_physical_devices('GPU')
for device in gpu_devices:
    tf.config.experimental.set_memory_growth(device, True)
from sklearn.preprocessing import StandardScaler, MinMaxScaler


In [2]:
df = pd.read_pickle('./final.pkl')

In [3]:
mb_am_temp = 56
mb_af_temp = 63
mb_pm_temp = 60

In [4]:
# Function to map AMafPM to ref_temp
def map_to_ref_temp(row):
    if row['AMafPM'] == [1,0,0]:
        return 56
    elif row['AMafPM'] == [0,1,0]:
        return 63
    elif row['AMafPM'] == [0,0,1]:
        return 60
    else:
        return None  # or a default value

# Apply the function to create the new column
df['ref_temp'] = df.apply(map_to_ref_temp, axis=1)

In [5]:
landuse_dim, water_dim, natural_dim, highway_dim  = [len(df.iloc[0,:]['feature_grids'][x][0][0]) for x in df.iloc[0,:]['feature_grids'].keys()]

In [6]:
height_input = Input(shape=(64, 64, 1), name='height')
landuse_input = Input(shape=(64, 64, landuse_dim), name='landuse')
water_input = Input(shape=(64, 64, water_dim), name='water')
natural_input = Input(shape=(64, 64, natural_dim), name='natural')
highway_input = Input(shape=(64, 64, highway_dim), name='highway')
time_input = Input(shape=(3,), name='time_of_day')  # Assuming 3 time periods: AM, Afternoon, PM
ref_temp_input = Input(shape=(1,), name='ref_temp')

# Example CNN Layer for each input (adjust as needed)
def create_cnn_block(input_layer, filters):
    x = Conv2D(filters, (3, 3), activation='relu')(input_layer)
    x = MaxPooling2D((2, 2))(x)
    x = Conv2D(filters, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    return x

In [7]:
# Creating CNN blocks for each spatial input
height_cnn = create_cnn_block(height_input, 32)
landuse_cnn = create_cnn_block(landuse_input, 32)
water_cnn = create_cnn_block(water_input, 32)
natural_cnn = create_cnn_block(natural_input, 32)
highway_cnn = create_cnn_block(highway_input, 32)

In [8]:
# Concatenating all the CNN outputs
concatenated = concatenate([Flatten()(height_cnn), 
                            Flatten()(landuse_cnn), 
                            Flatten()(water_cnn), 
                            Flatten()(natural_cnn), 
                            Flatten()(highway_cnn),
                            time_input,
                            ref_temp_input])  # Adding time input here

In [9]:
# Dense layers (adjust as needed)
x = Dense(256, activation='relu')(concatenated)
x = Dense(128, activation='relu')(x)

# Reshape to get a grid structure back
x = Dense(64*64, activation='relu')(x)
x = Reshape((64, 64, 1))(x)

# Upsample (if needed, depending on the size of the output from the CNN blocks)
# x = UpSampling2D(size=(2, 2))(x)

# Output layer to get the final 64x64 grid
output = Conv2D(1, (3, 3), activation='linear', padding='same')(x)

# Create Model
model = Model(inputs=[height_input, landuse_input, water_input, natural_input, highway_input, time_input,ref_temp_input], 
              outputs=output)

# Compile Model
model.compile(optimizer='adam', loss='mse')  # Use mean squared error as loss function

# Model Summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 height (InputLayer)            [(None, 64, 64, 1)]  0           []                               
                                                                                                  
 landuse (InputLayer)           [(None, 64, 64, 10)  0           []                               
                                ]                                                                 
                                                                                                  
 water (InputLayer)             [(None, 64, 64, 4)]  0           []                               
                                                                                                  
 natural (InputLayer)           [(None, 64, 64, 7)]  0           []                           

                                                                                                  
 conv2d_10 (Conv2D)             (None, 64, 64, 1)    10          ['reshape[0][0]']                
                                                                                                  
Total params: 8,643,754
Trainable params: 8,643,754
Non-trainable params: 0
__________________________________________________________________________________________________


In [10]:
df['landuse']=df['feature_grids'].apply(lambda x: x['landuse'])
df['water']=df['feature_grids'].apply(lambda x: x['water'])
df['natural']=df['feature_grids'].apply(lambda x: x['natural'])
df['highway']=df['feature_grids'].apply(lambda x: x['highway'])


In [11]:
scaler = MinMaxScaler()

# Find global min and max
global_min = np.min([np.min(subgrid) for subgrid in df["Subgrid"]])
global_max = np.max([np.max(subgrid) for subgrid in df["Subgrid"]])

# Fit the scaler using global min and max
scaler.fit(np.array([[global_min], [global_max]]))

# Apply the same scaling to each subgrid
scaled_subgrids = []

for subgrid in df["Subgrid"]:
    scaled_subgrid = scaler.transform(subgrid.flatten().reshape(-1, 1))
    scaled_subgrids.append(scaled_subgrid.reshape(subgrid.shape))

# Assign scaled subgrids back to DataFrame
df["Subgrid"] = pd.Series(scaled_subgrids, index=df.index)

In [12]:
def rotate_and_augment(df):
    # Creating an empty DataFrame to store augmented data
    augmented_data = pd.DataFrame()
    
    # Iterating through each row in the original DataFrame
    for _, row in df.iterrows():
        # Storing original and rotated versions of each grid in a list
        all_versions = []
        
        # Original data
        original_data = {
            "Subgrid": row["Subgrid"],
            "height_grid": row["height_grid"],
            "landuse": row["landuse"],
            "water": row["water"],
            "natural": row["natural"],
            "highway": row["highway"],
            # ... include other non-rotated columns as needed
            "AMafPM": row["AMafPM"],
            "ref_temp": row["ref_temp"]
        }
        all_versions.append(original_data)
        
        # Creating rotated versions
        for angle in [90, 180, 270]:
            rotated_data = {
                "Subgrid": np.rot90(row["Subgrid"], k=angle//90),
                "height_grid": np.rot90(row["height_grid"], k=angle//90),
                "landuse": np.rot90(row["landuse"], k=angle//90, axes=(0, 1)),
                "water": np.rot90(row["water"], k=angle//90, axes=(0, 1)),
                "natural": np.rot90(row["natural"], k=angle//90, axes=(0, 1)),
                "highway": np.rot90(row["highway"], k=angle//90, axes=(0, 1)),
                # ... include other non-rotated columns as needed
                "AMafPM": row["AMafPM"],
                "ref_temp": row["ref_temp"]
            }
            all_versions.append(rotated_data)
        
        # Appending all versions to the augmented_data DataFrame
        augmented_data = augmented_data.append(all_versions, ignore_index=True)
    
    return augmented_data
df = rotate_and_augment(df)

/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_1

/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_1

/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_10298/3210778621.py:40: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  augmented_data = augmented_data.append(all_versions, ignore_index=True)
/tmp/ipykernel_1

In [13]:
from sklearn.model_selection import train_test_split
test_size_ratio = 0.2  # Here, 20% of the data goes to the test set.

# Split the data
train_df, test_df = train_test_split(df, test_size=test_size_ratio, random_state=42)

In [14]:
height_grid = np.array(train_df['height_grid'].tolist())
landuse = np.array(train_df['landuse'].tolist())
water = np.array(train_df['water'].tolist())
natural = np.array(train_df['natural'].tolist())
highway = np.array(train_df['highway'].tolist())
AMafPM = np.array(train_df['AMafPM'].tolist())
ref_temp = np.array(train_df['ref_temp'].tolist())
target = np.array(train_df['Subgrid'].tolist())

In [15]:
height_grid_test = np.array(test_df['height_grid'].tolist())
landuse_test = np.array(test_df['landuse'].tolist())
water_test = np.array(test_df['water'].tolist())
natural_test = np.array(test_df['natural'].tolist())
highway_test = np.array(test_df['highway'].tolist())
AMafPM_test = np.array(test_df['AMafPM'].tolist())
ref_temp_test = np.array(test_df['ref_temp'].tolist())
target_test = np.array(test_df['Subgrid'].tolist())

In [16]:
#df['ref_temp'] = (df['ref_temp'] - df['ref_temp'].mean()) / df['ref_temp'].std()

In [17]:
%load_ext tensorboard
from datetime import datetime
from packaging import version

import tensorflow as tf
from tensorflow import keras

print("TensorFlow version: ", tf.__version__)
assert version.parse(tf.__version__).release[0] >= 2, \
    "This notebook requires TensorFlow 2.0 or above."
import tensorboard
tensorboard.__version__
logdir="logs/fit/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=logdir)


TensorFlow version:  2.9.2


In [18]:
train = model.fit([height_grid, landuse, water, natural, highway, AMafPM, ref_temp],
                    target,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                 callbacks=[tensorboard_callback])  # Adjust as needed

Epoch 1/100
51/51 [==============================] - 4s 29ms/step - loss: 0.1883 - val_loss: 0.1704
Epoch 2/100
51/51 [==============================] - 1s 14ms/step - loss: 0.1524 - val_loss: 0.1411
Epoch 3/100
51/51 [==============================] - 1s 14ms/step - loss: 0.1244 - val_loss: 0.1141
Epoch 4/100
51/51 [==============================] - 1s 14ms/step - loss: 0.0872 - val_loss: 0.0589
Epoch 5/100
51/51 [==============================] - 1s 14ms/step - loss: 0.0396 - val_loss: 0.0299
Epoch 6/100
51/51 [==============================] - 1s 15ms/step - loss: 0.0237 - val_loss: 0.0219
Epoch 7/100
51/51 [==============================] - 1s 14ms/step - loss: 0.0190 - val_loss: 0.0183
Epoch 8/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0161 - val_loss: 0.0164
Epoch 9/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0145 - val_loss: 0.0155
Epoch 10/100
51/51 [==============================] - 1s 14ms/step - loss: 0.0134 - val_loss: 0.0144

51/51 [==============================] - 1s 13ms/step - loss: 0.0046 - val_loss: 0.0062
Epoch 83/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0046 - val_loss: 0.0062
Epoch 84/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0046 - val_loss: 0.0063
Epoch 85/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0046 - val_loss: 0.0061
Epoch 86/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0046 - val_loss: 0.0063
Epoch 87/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0046 - val_loss: 0.0062
Epoch 88/100
51/51 [==============================] - 1s 14ms/step - loss: 0.0046 - val_loss: 0.0062
Epoch 89/100
51/51 [==============================] - 1s 14ms/step - loss: 0.0046 - val_loss: 0.0062
Epoch 90/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0046 - val_loss: 0.0063
Epoch 91/100
51/51 [==============================] - 1s 13ms/step - loss: 0.0044 - val_loss: 0.0063
Epo

In [19]:
x = model.evaluate(x=[height_grid_test, landuse_test, water_test, natural_test, highway_test, AMafPM_test,ref_temp_test], y=target_test)


16/16 [==============================] - 0s 11ms/step - loss: 0.0056


In [20]:
x

0.005643698386847973

In [21]:
normalized_mse = x
original_mse = normalized_mse * (global_max - global_min)

# Optionally, compute RMSE
original_rmse = np.sqrt(original_mse)

# Display
print(f"Original Scale MSE: {original_mse}")
print(f"Original Scale RMSE: {original_rmse}")


Original Scale MSE: 0.12040808462643326
Original Scale RMSE: 0.34699868101540854


In [22]:
%tensorboard --logdir logs --host localhost
